In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import math

#Algoritimos

## Branch and Bound

In [ ]:
class Ponto:
  def __init__(self, x, y):
    self.x = x
    self.y = y

class Node:
    def __init__(self, bound, boundEdges, cost, solution):
        self.bound = bound
        self.boundEdges = boundEdges
        self.cost = cost
        self.solution = solution
    def __lt__(self, other):
        if len(self.solution) == len(other.solution):
            return self.bound < other.bound
        return len(self.solution) > len(other.solution)


def findTwoMinimalEdges(list):
    min1 = np.inf
    min2 = np.inf
    for j in list:
        if list[j]['weight'] < min1:
            min2 = min1
            min1 = list[j]['weight']
        elif list[j]['weight'] < min2:
            min2 = list[j]['weight']
    return min1, min2

def findInitialBound(A):
    bound = 0
    initialBoundEdges = np.zeros((A.number_of_nodes(), 2), dtype=list)
    for i in range(A.number_of_nodes()):
        min1, min2 = findTwoMinimalEdges(A[i])
        initialBoundEdges[i][0] = min1
        initialBoundEdges[i][1] = min2
        bound += min1 + min2
    del min1, min2
    return bound / 2, initialBoundEdges

def findBound(A, solution, boundEdges, bound):
    changedEdges = np.zeros(A.number_of_nodes(), dtype=int)
    newEdges = np.array(boundEdges)
    edgeWeight = A[solution[-2]][solution[-1]]['weight']
    sum = bound * 2
    if newEdges[solution[-2]][0] != edgeWeight:
        if changedEdges[solution[-2]] == 0:
            sum -= newEdges[solution[-2]][1]
            sum += edgeWeight
        else:
            sum -= newEdges[solution[-2]][0]
            sum += edgeWeight
        changedEdges[solution[-2]] += 1
    if newEdges[solution[-1]][0] != edgeWeight:
        if changedEdges[solution[-1]] == 0:
            sum -= newEdges[solution[-1]][1]
            sum += edgeWeight
        else:
            sum -= newEdges[solution[-1]][0]
            sum += edgeWeight
        changedEdges[solution[-1]] += 1
    del changedEdges, edgeWeight
    return sum / 2, newEdges

In [ ]:
from heapq import heappush, heappop

def branchAndBoundTSP(A):
    initialBound, initialBoundEdges = findInitialBound(A)
    root = Node(initialBound, initialBoundEdges, 0, [0])
    heap = []
    heappush(heap, root)
    del root, initialBoundEdges, initialBound
    best = np.inf
    solution = []
    nodeCount = 0
    while heap:
        node = heappop(heap)
        nodeCount += 1
        level = len(node.solution)
        if level > A.number_of_nodes():
            if best > node.cost:
                best = node.cost
                solution = node.solution
        else:
            if node.bound < best:
                if level < A.number_of_nodes() - 2:
                    for k in range(1, A.number_of_nodes()):
                        if k == node.solution[-1] or k == 0:
                            continue
                        edgeWeight = A[node.solution[-1]][k]['weight']
                        newBound, newEdges = findBound(A, node.solution + [k], node.boundEdges, node.bound)
                        if k not in node.solution and newBound < best:
                            newNode = Node(newBound, newEdges, node.cost + edgeWeight, node.solution + [k])
                            if k == 2:
                                if 1 not in node.solution:
                                    continue
                            heappush(heap, newNode)
                else:
                    for k in range(1, A.number_of_nodes()):
                        if k == node.solution[-1] or k == 0:
                            continue
                        lastNode = 0
                        for i in range(1, A.number_of_nodes()):
                            if i not in node.solution + [k] and k != i:
                                lastNode = i
                                break
                        edgeWeight = A[node.solution[-1]][k]['weight']
                        nextEdgeWeight = A[k][lastNode]['weight']
                        lastEdgeWeight = A[lastNode][0]['weight']
                        cost = node.cost + edgeWeight + nextEdgeWeight + lastEdgeWeight
                        if k not in node.solution and cost < best:
                            newNode = Node(cost, [], cost, node.solution + [k, lastNode, 0])
                            heappush(heap, newNode)
    del heap, nodeCount
    return best, solution


## Twice Around The Tree

In [ ]:
def findPathWeight(A, path):
    weight = 0
    for i in range(len(path) - 1):
        weight += A[path[i]][path[i + 1]]['weight']
    return weight

def twiceAroundTheTreeTSP(A):
    MST = nx.minimum_spanning_tree(A)
    path = nx.dfs_preorder_nodes(MST, 0)
    del MST
    path = list(path)
    path.append(path[0])
    weight = findPathWeight(A, path)
    return weight, path


## Christofides

In [ ]:
def findShortcutPath(A):
    path = list(nx.eulerian_circuit(A, 0))
    path = [x[0] for x in path]

    shortcutPath = list(dict.fromkeys(path))

    return shortcutPath + [shortcutPath[0]]

def christofidesTSP(A):
    MST = nx.minimum_spanning_tree(A)
    degrees = nx.degree(MST)
    oddNodes = [x[0] for x in degrees if degrees[x[0]] % 2 == 1]
    del degrees
    oddNodesSubgraph = nx.subgraph(A, oddNodes)
    matching = list(nx.min_weight_matching(oddNodesSubgraph))
    del oddNodesSubgraph

    MSTMultiGraph = nx.MultiGraph(MST)
    del MST
    for i in range(len(matching)):
        node1 = matching[i][0]
        node2 = matching[i][1]
        MSTMultiGraph.add_edge(node1, node2, weight=A[node1][node2]['weight'])
    del matching

    path = findShortcutPath(MSTMultiGraph)
    weight = findPathWeight(A, path)

    return weight, path


## Testando

In [ ]:
def readLine(file):
  return file.readline().strip().split()[1]

def getPoints(infile, Dimension):
  listOfPoints = []
  size = int(Dimension)

  for i in range(size):
    x,y = infile.readline().strip().split()[1:]
    listOfPoints.append(Ponto(float(x),float(y)))

  return listOfPoints

def getEuclidianDistance(point1, point2):
  return math.sqrt((point1.x-point2.x)**2 + (point1.y-point2.y)**2)

def getNodesDistance(list):
  nodesDistance = nx.complete_graph(len(list))
  for i in range(len(list)):
    for j in range(len(list)):
      if list[i] != list[j]:
        nodesDistance[i][j]['weight'] = getEuclidianDistance(list[i],list[j])
  return nodesDistance

In [ ]:
import time
from functools import wraps

def timeout(seconds):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            start_time = time.time()
            result = func(*args, **kwargs)
            elapsed_time = time.time() - start_time
            if elapsed_time > seconds:
                raise TimeoutError(f"A execução excedeu o tempo limite de {seconds} segundos.")
            return result
        return wrapper
    return decorator

### Timeout e criando as instâncias

In [ ]:
import time

@timeout(1800)
def testBranchAndBoundTime(nodesListWithDistance):
    start = time.time()
    weight, path = branchAndBoundTSP(nodesListWithDistance)
    end = time.time()

    return weight, path, end - start
    pass

@timeout(1800)
def testTwiceAroundTheTreeTSP(nodesListWithDistance):
    start = time.time()
    weight, path = twiceAroundTheTreeTSP(nodesListWithDistance)
    end = time.time()

    return weight, path, end - start
    pass

@timeout(1800)
def testChristofidesTSP(nodesListWithDistance):
    start = time.time()
    weight, path = christofidesTSP(nodesListWithDistance)
    end = time.time()

    return weight, path, end - start
    pass

fileList = [
    "eil51",
    "berlin52",
    "st70",
    "eil76",
    "pr76",
    "rat99",
    "kroA100",
    "kroB100",
    "kroC100",
    "kroD100",
    "kroE100",
    "rd100",
    "eil101",
    "lin105",
    "pr107",
    "pr124",
    "bier127",
    "ch130",
    "pr136",
    "pr144",
    "ch150",
    "kroA150",
    "kroB150",
    "pr152",
    "u159",
    "rat195",
    "d198",
    "kroA200",
    "kroB200",
    "ts225",
    "tsp225",
    "pr226",
    "gil262",
    "pr264",
    "a280",
    "pr299",
    "lin318",
    "linhp318",
    "rd400",
    "fl417",
    "pr439",
    "pcb442",
    "d493",
    "u574",
    "rat575",
    "p654",
    "d657",
    "u724",
    "rat783",
    "pr1002",
    "u1060",
    "vm1084",
    "pcb1173",
    "d1291",
    "rl1304",
    "rl1323",
    "nrw1379",
    "fl1400",
    "u1432",
    "fl1577",
    "d1655",
    "vm1748",
    "u1817",
    "rl1889",
    "d2103",
    "u2152",
    "u2319",
    "pr2392",
    "pcb3038",
    "fl3795",
    "fnl4461",
    "rl5915",
    "rl5934",
    "rl11849",
    "usa13509",
    "brd14051",
    "d15112",
    "d18512",
]

costExpected = [
    426,
    7542,
    675,
    538,
    108159,
    1211,
    21282,
    22141,
    20749,
    21294,
    22068,
    7910,
    629,
    14379,
    44303,
    59030,
    118282,
    6110,
    96772,
    58537,
    6528,
    26524,
    26130,
    73682,
    42080,
    2323,
    15780,
    29368,
    29437,
    126643,
    3919,
    80369,
    2378,
    49135,
    2579,
    48191,
    42029,
    41345,
    15281,
    11861,
    107217,
    50778,
    35002,
    36905,
    6773,
    34643,
    48912,
    41910,
    8806,
    259045,
    224094,
    239297,
    56892,
    50801,
    252948,
    270199,
    56638,
    20127,
    152970,
    22249,
    62128,
    336556,
    57201,
    316536,
    80450,
    64253,
    234256,
    378032,
    137694,
    28772,
    182566,
    565530,
    556045,
    923368,
    19982889,
    469445,
    1573152,
    645488,
]


###Calculando o fator de Aproximação

In [ ]:
def getApproximation(otimo, obtido):
  return obtido/otimo

def runTwice():
  for item in range(len(fileList)):
    twiceFile = open("twiceAroundTheTreeTSP.txt", "a")
    nodesListWithDistance = []
    print("Running:", fileList[item])
    infile = open(fileList[item]+".tsp", 'r')
    Dimension = ""
    while(Dimension.isnumeric() == False):
      line = infile.readline()
      if(line.split()[1] != ':'):
        Dimension = line.split()[1]
      else:
        Dimension = line.split()[2]
    print(Dimension)
    line = ""
    while(line != "NODE_COORD_SECTION\n"):
      line = infile.readline()
    del line

    pointsList = getPoints(infile, Dimension)

    infile.close()

    nodesListWithDistance = getNodesDistance(pointsList)

    twiceFile.write("Running: " + fileList[item] + "\n")

    del pointsList
    del Dimension

    twiceFile.write("Expected: " + str(costExpected[item]) + "\n")
    weight, solution, timeElapsed = testTwiceAroundTheTreeTSP(nodesListWithDistance)
    del nodesListWithDistance
    twiceFile.write("TwiceAroundTheTreeTSP: " + str(weight) + "\n")
    twiceFile.write("Aproximação obtida: " + str(getApproximation(weight, costExpected[item])) + "\n")
    twiceFile.write("Validade da aproximação:" + str(weight <= 2 * costExpected[item] and weight >= costExpected[item]) + "\n")
    twiceFile.write("Tempo: " + str(timeElapsed) + "\n")

    del weight, solution, timeElapsed

    twiceFile.write("\n")

    twiceFile.close()

def runChris():
  for item in range(len(fileList)):
    chrisFile = open("christofidesTSP.txt", "a")
    nodesListWithDistance = []
    print("Running:", fileList[item])
    infile = open(fileList[item]+".tsp", 'r')
    Dimension = ""
    while(Dimension.isnumeric() == False):
      line = infile.readline()
      if(line.split()[1] != ':'):
        Dimension = line.split()[1]
      else:
        Dimension = line.split()[2]
    line = ""
    while(line != "NODE_COORD_SECTION\n"):
      line = infile.readline()
    del line

    pointsList = getPoints(infile, Dimension)

    infile.close()

    nodesListWithDistance = getNodesDistance(pointsList)

    chrisFile.write("Running: " + fileList[item] + "\n")

    del pointsList
    del Dimension

    chrisFile.write("Expected: " + str(costExpected[item]) + "\n")
    weight, solution, timeElapsed = testChristofidesTSP(nodesListWithDistance)
    del nodesListWithDistance
    chrisFile.write("ChristofidesTSP: " + str(weight) + "\n")
    chrisFile.write("Aproximação obtida: " + str(getApproximation(weight, costExpected[item])) + "\n")
    chrisFile.write("Validade da aproximação:" + str(weight <= 2 * costExpected[item] and weight >= costExpected[item]) + "\n")
    chrisFile.write("Tempo: " + str(timeElapsed) + "\n")

    chrisFile.write("\n")
    chrisFile.close()

def runBranch():
  for item in range(len(fileList)):
    branchFile = open("branchAndBound.txt", "a")
    nodesListWithDistance = []
    print("Runing:", fileList[item])
    infile = open(fileList[item]+".tsp", 'r')
    Dimension = ""
    while(Dimension.isnumeric() == False):
      line = infile.readline()
      if(line.split()[1] != ':'):
        Dimension = line.split()[1]
      else:
        Dimension = line.split()[2]
    line = ""
    while(line != "NODE_COORD_SECTION\n"):
      line = infile.readline()
    del line

    pointsList = getPoints(infile, Dimension)

    infile.close()

    nodesListWithDistance = getNodesDistance(pointsList)

    branchFile.write("Runing: " + fileList[item] + "\n")

    del pointsList
    del Dimension

    branchFile.write("Expected: " + str(costExpected[item]) + "\n")
    weight, solution, timeElapsed = testBranchAndBoundTime(nodesListWithDistance)
    del nodesListWithDistance
    branchFile.write("Branch and Bound: " + str(weight) + "\n")
    branchFile.write("Aproximação obtida: " + str(getApproximation(weight, costExpected[item])) + "\n")
    branchFile.write("Validade da aproximação:" + str(weight == costExpected[item]) + "\n")
    branchFile.write("Tempo: " + str(timeElapsed) + "\n")

    del weight, solution, timeElapsed
    branchFile.write("\n")

    branchFile.close()


###Execução

In [ ]:
def twiceRunning():
  try:
    runTwice()
  except TimeoutError as e:
    print(e)
    twiceFile = open("twiceAroundTheTreeTSP.txt", "a")
    twiceFile.write(e+"\n")
    twiceFile.close()

def chrisRunning():
  try:
    runChris()
  except TimeoutError as e:
    print(e)
    twiceFile = open("christofidesTSP.txt", "a")
    twiceFile.write(e+"\n")
    twiceFile.close()

def branchRunning():
  try:
    runBranch()
  except TimeoutError as e:
    print(e)
    twiceFile = open("branchandbound.txt", "a")
    twiceFile.write(e+"\n")
    twiceFile.close()

## Twice Around

In [ ]:
twiceRunning()

## Christofides

In [ ]:
chrisRunning()

## Branch

In [ ]:
branchRunning()